In [0]:
-- Aggregate marketing spend to the base S12N, month level
DROP TABLE IF EXISTS dev.bi.s12n_marketing_spend_aggregated_mgtv;
CREATE TABLE dev.bi.s12n_marketing_spend_aggregated_mgtv AS
(
WITH marketing_spend AS
(
  SELECT 
      spend_month,
      launch_month,
      base_phoenix_specialization_id,
      SUM(marketing_spend) AS marketing_spend
  FROM 
      dev.bi.s12n_marketing_spend_mgtf
  WHERE spend_month IS NOT NULL
  GROUP BY 1, 2, 3
),

-- Join this with the date sequence table
filled_marketing_data AS
(
SELECT 
    COALESCE(ms.spend_month, ds.date_month) AS date_month,
    COALESCE(ms.launch_month, ds.start_month) AS launch_month,
    COALESCE(ms.base_phoenix_specialization_id, ds.base_phoenix_specialization_id) AS base_phoenix_specialization_id,
    COALESCE(ms.marketing_spend, 0) AS marketing_spend
FROM 
    marketing_spend ms
RIGHT JOIN
    dev.bi.s12n_date_sequence_mgtv ds -- any marketing spend prior to launch is not captured. Reasonable to assume that marketing will not be done till launch?
ON 
    ms.base_phoenix_specialization_id = ds.base_phoenix_specialization_id
AND
    ms.spend_month = ds.date_month
)

-- Add marketing spend aggregate from the current month to the next 11 months (12 months total)
-- The n12m value when spend_month = launch_month is the Y1 marketing spend
SELECT 
    date_month,
    launch_month,
    base_phoenix_specialization_id,
    marketing_spend,
    SUM(marketing_spend) OVER (PARTITION BY base_phoenix_specialization_id ORDER BY date_month
                               ROWS BETWEEN CURRENT ROW AND 11 FOLLOWING) AS rolling_n12m_marketing_spend
FROM 
filled_marketing_data
)

num_affected_rows,num_inserted_rows


# QA

In [0]:
-- select * from dev.bi.s12n_marketing_spend_aggregated_mgtv where base_phoenix_specialization_id = '--I-GVG8TjSxWxqmqcbomg'
-- select * from dev.general.s12n_marketing_spend_aggregated where base_phoenix_specialization_id = '--I-GVG8TjSxWxqmqcbomg'

date_month,launch_month,base_phoenix_specialization_id,marketing_spend,rolling_n12m_marketing_spend
2024-10-01,2024-10-01,--I-GVG8TjSxWxqmqcbomg,0.0,2095926.835742
2024-11-01,2024-10-01,--I-GVG8TjSxWxqmqcbomg,478121.2914879997,2095926.835742
2024-12-01,2024-10-01,--I-GVG8TjSxWxqmqcbomg,272359.0383360002,1617805.5442540003
2025-01-01,2024-10-01,--I-GVG8TjSxWxqmqcbomg,353109.68441599986,1345446.5059180001
2025-02-01,2024-10-01,--I-GVG8TjSxWxqmqcbomg,233018.38038400005,992336.8215020002
2025-03-01,2024-10-01,--I-GVG8TjSxWxqmqcbomg,284991.3334660001,759318.4411180002
2025-04-01,2024-10-01,--I-GVG8TjSxWxqmqcbomg,212579.90901999996,474327.10765200015
2025-05-01,2024-10-01,--I-GVG8TjSxWxqmqcbomg,261747.19863200016,261747.19863200016
2025-06-01,2024-10-01,--I-GVG8TjSxWxqmqcbomg,0.0,0.0
